In [63]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

import pandas as pd

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

train.shape, test.shape

((8101, 21), (2026, 20))

In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 8101 non-null   int64  
 1   Customer_Age              8101 non-null   int64  
 2   Gender                    8101 non-null   object 
 3   Dependent_count           8101 non-null   int64  
 4   Education_Level           8101 non-null   object 
 5   Marital_Status            8101 non-null   object 
 6   Income_Category           8101 non-null   object 
 7   Card_Category             8101 non-null   object 
 8   Months_on_book            8101 non-null   int64  
 9   Total_Relationship_Count  8101 non-null   int64  
 10  Months_Inactive_12_mon    8101 non-null   int64  
 11  Contacts_Count_12_mon     8101 non-null   int64  
 12  Credit_Limit              8101 non-null   float64
 13  Total_Revolving_Bal       8101 non-null   int64  
 14  Avg_Open

In [65]:
train["Attrition_Flag"].value_counts()

Attrition_Flag
0    6815
1    1286
Name: count, dtype: int64

In [66]:
# baseline 
cols = train.select_dtypes(include='object').columns
cols

train.drop(cols, axis=1).shape

train = train.select_dtypes(exclude="object")
test = test.select_dtypes(exclude="object")

print(train.shape) 
print(test.shape)


(8101, 16)
(2026, 15)


In [67]:
train = train.drop("CLIENTNUM", axis=1)
test_id = test.pop("CLIENTNUM")

In [68]:
train.head(1)

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,38,1,23,4,3,2,22149.0,987,21162.0,1.633,2409,36,0.8,0.045,0


In [69]:
test.head(1)

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,43,2,37,4,3,0,6611.0,1041,5570.0,0.744,4037,74,0.682,0.157


In [70]:
test_id

0       788544108
1       719356008
2       712142733
3       789394833
4       742778358
          ...    
2021    787361583
2022    713474733
2023    708486033
2024    717683583
2025    712887633
Name: CLIENTNUM, Length: 2026, dtype: int64

In [71]:
# 검증데이터 분리
from sklearn.model_selection import train_test_split 

# drop() axix=0 행을 기준으로 삭제, axis=1 열을 기준으로 삭제
X_tr, X_val, y_tr, y_val = train_test_split(
  train.drop("Attrition_Flag", axis=1), 
  train["Attrition_Flag"], 
  test_size=0.2, 
  random_state=2022

)

In [72]:
X_tr.head()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
6806,50,1,30,1,3,2,14217.0,1816,12401.0,0.978,15554,99,0.678,0.128
5635,50,2,36,5,4,1,2428.0,2005,423.0,0.835,5170,68,0.659,0.826
2517,43,5,32,2,3,1,11508.0,0,11508.0,1.005,5242,74,0.574,0.000
2717,47,3,36,4,2,2,2926.0,0,2926.0,0.676,1304,24,0.846,0.000
3773,63,0,53,5,3,2,2271.0,1278,993.0,0.375,1550,33,0.650,0.563


In [73]:
# 모델 & 평가 
# 모델 불러오기 
# 학습 
# 예측 
from sklearn.ensemble import RandomForestClassifier 

model = RandomForestClassifier() 

model.fit(X_tr, y_tr) 
pred = model.predict(X_val)

In [74]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [75]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score 

# 정확도 
print(accuracy_score(y_val, pred))

# 정밀도
print(precision_score(y_val, pred))

# 재현율(민감도)
print(recall_score(y_val, pred))

# F1 
print(f1_score(y_val, pred))

0.9673041332510796
0.9563318777292577
0.8358778625954199
0.8920570264765784


In [76]:
# roc-auc 
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:, 1]))

# baseline
from sklearn.preprocessing import LabelEncoder

for col in cols: 
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

train.head()

0.9895761364721478


KeyError: 'Gender'